In [1]:
from ipynb.fs.full.Adversarial_NIDS_Environemnt import *

C:\Users\rahul\Downloads\Adversarial NIDS Payload Level\Phase 4\Modulated Code\Adversarial_NIDS_Environemnt.ipynb:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  },


In [2]:
from ipynb.fs.full.DQN_agent import *

In [3]:
import time
start_time = time.time()

list_win_experiences = []
win_count = 0
all_epi_rewards = []
print_reward = 0
payload_big = payload_1 + payload_1

for episode in range(num_episodes):
    
    list_experiences = []
    state_ = df.iloc[random.choice(list_ind)]
    state_array = state_extractor(state_).to_numpy()
    state_tensor = torch.FloatTensor(state_array).to(device)
    payload_no = 0
    epi_reward = 0
    if episode % 1000 == 0:
        
        if episode % 5000 == 0:
            print("Episode",episode,'      ',time.time()-start_time, '        ', print_reward/5000)
            print_reward = 0
        else:
            print("Episode",episode,'      ',time.time()-start_time)
            
    for timestep in range(episode_length):
        transition_list = []
#         print('State', state_array)
        action_ = agent.select_action(state_,state_tensor, policy_net)
        action = torch.tensor([int(action_)],dtype = torch.int64).to(device)
        if action_[0] == 10:
            payload_no += 20
            payload = payload_big[payload_no:payload_no+20]
        else:
            payload = 0
        
#         print("Action", action)
#         print(action)
        next_state_, reward, done = step(state_,int(action),payload)
        epi_reward += reward
        if done:
            next_state_array = final_state_df.to_numpy()
        else:
            next_state_array = state_extractor(next_state_).to_numpy()
#         print("Reward",reward)
        next_state_tensor = torch.FloatTensor(next_state_array).to(device)
        
        reward_tensor = torch.FloatTensor([reward]).to(device)
        
        memory.push(Experience(state_tensor, action, next_state_tensor, reward_tensor))
        
        state_ = next_state_
        if done != True:
            state_array = state_extractor(state_).to_numpy()
    
        state_tensor = next_state_tensor
        
        transition_list.append(state_array)
        transition_list.append(int(action))
        transition_list.append(reward)
        transition_list.append(next_state_array)
        list_experiences.append(transition_list)
        
        if done:
            list_win_experiences.append(list_experiences)
            win_count += 1
            break
    
        
        if memory.can_provide_sample(batch_size):
            experiences = memory.sample(batch_size)
            states, actions, rewards, next_states = extract_tensors(experiences)
            current_q_values = QValues.get_current(policy_net, states, actions)
            next_q_values = QValues.get_next(target_net, next_states)
            target_q_values = (next_q_values * gamma) + rewards
            loss = F.huber_loss(current_q_values, target_q_values.unsqueeze(1),reduction='mean', delta=1.0)
#             torch.nn.HuberLoss(reduction='mean', delta=1.0)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
    print_reward += epi_reward
    all_epi_rewards.append(epi_reward)
    if episode % target_update == 0:
        target_net.load_state_dict(policy_net.state_dict())
end_time = time.time()

Episode 0        0.0448603630065918          0.0


KeyboardInterrupt: 

In [ ]:
torch.save(policy_net.state_dict(), 'Agent_DQN_portscanagent_ensemble.pkl')

In [ ]:
## Saving all obtained rewards for all training episodes
np.savetxt("Agent_DQN_Allrewards_portscanagent_dosclass.csv",all_epi_rewards,delimiter=", ",fmt='% s')

In [ ]:
# Counter(all_epi_rewards)

In [ ]:
inf = 0
for i in range(len(list_win_experiences)):
    if list_win_experiences[-i][-1][-2] == -1000:
        inf += 1

In [ ]:
inf

In [ ]:
win_count = win_count-inf

In [ ]:
## Obtain the number of wins
wincount

In [ ]:
## Obtain Runtime
run_time = end_time - start_time 
run_time